## Introduction to Route Analysis using Batfish

Network engineers routinely need to validate routing and forwarding in the network. They often do that by connecting to multiple network devices and executing a series of `show route` commands. This distributed debugging is highly complex even in a moderately-sized network. Batfish makes this task extremely simple by providing an easy-to-query, centralized view of routing tables in the network. 

In this notebook, we will look at how you can extract routing information from Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-route-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py

 ### Initializing the Network and Snapshot

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory, see the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.<br>
More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [2]:
# Initialize a network and snapshot
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

The network snapshot that we initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/example).

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

### Viewing Routing Tables
Batfish makes **all** routing tables in the network easily accessible.

In [3]:
# Get routing tables for all nodes and VRFs
routes_all = bfq.routes().answer().frame()

We are not going to print this table as it has a large number of entries. To extract a subset of the entries, we can run the `routes()` question with parameters of interest.

In [4]:
# Get the routing table for the 'default' VRF on border routers of as1
routes_as1border = bfq.routes(nodeRegex="as1border.*", vrfRegex="default").answer().frame()
routes_as1border

,Node,VRF,Network,Protocol,Tag,NextHopIp,NextHop,AdminDistance,Metric
0,as1border1,default,10.13.22.0/24,ospfE2,-1,1.0.1.2,as1core1,110,20
1,as1border2,default,10.14.22.1/32,local,-1,AUTO/NONE(-1l),None,0,0
2,as1border1,default,3.0.2.0/24,ibgp,-1,10.13.22.3,as3border2,200,50
3,as1border2,default,10.12.11.0/24,ospfE2,-1,1.0.2.2,as1core1,110,20
4,as1border1,default,10.14.22.0/24,ospfE2,-1,1.0.1.2,as1core1,110,20
5,as1border1,default,10.12.11.1/32,local,-1,AUTO/NONE(-1l),None,0,0
6,as1border2,default,1.0.2.1/32,local,-1,AUTO/NONE(-1l),None,0,0
7,as1border1,default,2.128.0.0/16,bgp,-1,10.12.11.2,as2border1,20,50
8,as1border2,default,10.13.22.1/32,local,-1,AUTO/NONE(-1l),None,0,0
9,as1border2,default,2.128.0.0/16,ibgp,-1,10.12.11.2,as2border1,200,50


### Debugging Connectivity
To debug a connectivity issue, network engineers often need to analyze the routing entries for a prefix at multiple devices. The commands below show how to accomplish this task, using the prefix **1.0.2.0/24** as an example.

In [5]:
# Get routes for '1.0.2.0/24'
routes_filtered = routes_all[routes_all["Network"] == "1.0.2.0/24"]

# Get the unique node names (using Pandas) and sort the list
print(sorted(routes_filtered["Node"].unique()))

['as1border1', 'as1border2', 'as1core1', 'as2border1', 'as2border2', 'as2core1', 'as2core2', 'as2dept1', 'as2dist1', 'as2dist2', 'as3border1', 'as3border2', 'as3core1']


Listing which routers have routes to a given prefix is useful, but we may also want to know which routers do **not** have the prefix. We can easily accomplish that as well using the Pandas [groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) and [filter](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.filter.html) functions.

In [6]:
# Group all routes by Node and filter for those that don't have '1.0.2.0/24'
nodes_filtered = routes_all.groupby('Node').filter(lambda x: all(x['Network'] != '1.0.2.0/24'))

# Get the unique node names and sort the list
print(sorted(nodes_filtered["Node"].unique()))

['host1', 'host2']


In this case, we see that all of the routers have the prefix. The only nodes without the prefix are the two hosts, 'host1' and 'host2'.

This notebook showed you

1. How to get routes at all nodes in the network or only at a subset of them
2. How to find which nodes have an entry for a prefix or which ones do not

These are foundational tasks for route analysis. Future notebooks will dive into more advanced scenarios. Stay tuned!

### Want to know more? 

Reach out to us through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to learn more or send feedback.